Imports

In [253]:
from multiprocessing import freeze_support
from timed_decorator.simple_timed import timed

import torch
from torch import nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter

Load images as normalized tensors, discard labels

In [254]:
def get_cifar10_images(data_path: str, train: bool):
    initial_transforms = v2.Compose([
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True)
    ])
    cifar_10_images = CIFAR10(
        root=data_path,
        train=train,
        transform=initial_transforms,
        download=True
    )
    return [image for image, label in cifar_10_images]


 Custom Dataset Definition (input_image, transformed_image)

In [255]:
class CustomDataset(Dataset):
    def __init__(self, data_path: str = './data', train: bool = True, cache: bool = True):
        self.images = get_cifar10_images(data_path, train)
        self.cache = cache
        self.transforms = v2.Compose([
            v2.Resize((28, 28), antialias=True),
            v2.Grayscale(),
            v2.functional.hflip,
            v2.functional.vflip,
        ])
        if cache:
            self.labels = [self.transforms(x) for x in self.images]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, i):
        if self.cache:
            return self.images[i], self.labels[i]
        return self.images[i], self.transforms(self.images[i])


Measure time required to apply ground-truth transformations sequentially on CPU.

In [256]:
@timed(return_time=True, use_seconds=True, stdout=False)
def transform_dataset_with_transforms(dataset: TensorDataset):
    transforms = v2.Compose([
        v2.Resize((28, 28), antialias=True),
        v2.Grayscale(),
        v2.functional.hflip,
        v2.functional.vflip,
    ])
    for image in dataset.tensors[0]:
        transforms(image)


Model

In [257]:
class TinyTransformNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(3 * 32 * 32, 784),
            nn.Linear(784, 784),
            nn.Linear(784, 28 * 28)
        )

    def forward(self, x):
        b = x.size(0)
        x = x.reshape(b, -1)
        x = self.fc(x)
        return x.view(b, 1, 28, 28)


Placeholder for batched, model-based inference timing.

In [258]:
@timed(return_time=True, use_seconds=True, stdout=False)
@torch.no_grad()
def transform_dataset_with_model(dataset: TensorDataset, model: nn.Module, batch_size: int, device):
    model.eval()  # TODO: uncomment this
    model.to(device)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=(device.type == "cuda")
    )
    for (images,) in dataloader:
        images = images.to(device)
        model(images)


Compare CPU transforms vs model inference speed.

In [259]:
def test_inference_time(model: nn.Module, device=torch.device('cpu')):
    test_dataset = CustomDataset(train=False, cache=False)
    test_dataset = torch.stack(test_dataset.images)
    test_dataset = TensorDataset(test_dataset)

    batch_size = 100

    _, t1 = transform_dataset_with_transforms(test_dataset)
    _, t2 = transform_dataset_with_model(test_dataset, model, batch_size, device)

    print(
        f"Sequential transforming each image took: {t1}s on CPU.\n"
        f"Using a model with batch_size={batch_size} took {t2}s on {device.type}.\n"
    )


In [260]:
import time


def train_model(
    model,
    train_loader,
    val_loader,
    device,
    max_epochs=50,
    patience=5,
    writer=None,
):
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    best_val = float("inf")
    patience_counter = 0

    model.to(device)

    for epoch in range(max_epochs):
        epoch_start = time.time()
        model.train()
        train_loss = 0.0

        batch_times = []
        for x, y in train_loader:
            batch_start = time.time()

            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            batch_times.append(time.time() - batch_start)

        train_loss /= len(train_loader)
        avg_batch_time = sum(batch_times) / len(batch_times)

        model.eval()
        val_loss = 0.0
        val_batch_times = []
        with torch.no_grad():
            for x, y in val_loader:
                batch_start = time.time()
                x, y = x.to(device), y.to(device)
                pred = model(x)
                val_loss += criterion(pred, y).item()
                val_batch_times.append(time.time() - batch_start)

        val_loss /= len(val_loader)
        avg_val_batch_time = sum(val_batch_times) / len(val_batch_times)
        epoch_time = time.time() - epoch_start

        print(f"Epoch {epoch}: train={train_loss:.4f}, val={val_loss:.4f}, "
              f"epoch_time={epoch_time:.2f}s, avg_batch_time={avg_batch_time:.4f}s")

        if writer is not None:
            writer.add_scalar("Loss/Train", train_loss, epoch)
            writer.add_scalar("Loss/Validation", val_loss, epoch)
            writer.add_scalar("Time/Epoch", epoch_time, epoch)
            writer.add_scalar("Time/Train_Batch", avg_batch_time, epoch)
            writer.add_scalar("Time/Val_Batch", avg_val_batch_time, epoch)

        if val_loss < best_val:
            best_val = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "tiny_transform_net.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    if writer is not None:
        writer.close()


In [261]:
def get_dataloaders(batch_size=64):
    full_dataset = CustomDataset(train=True, cache=True)

    val_size = int(0.1 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    train_ds, val_ds = torch.utils.data.random_split(
        full_dataset, [train_size, val_size]
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0
    )

    return train_loader, val_loader

Device

In [262]:
train_loader, val_loader = get_dataloaders(batch_size=64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TinyTransformNet()
writer = SummaryWriter(log_dir="runs/tiny_transform_net")

train_model(
    model,
    train_loader,
    val_loader,
    device,
    max_epochs=100,
    patience=10,
    writer=writer,
)

Epoch 0: train=0.0795, val=0.0564, epoch_time=2.16s, avg_batch_time=0.0028s
Epoch 1: train=0.0543, val=0.0484, epoch_time=1.89s, avg_batch_time=0.0022s
Epoch 2: train=0.0432, val=0.0437, epoch_time=1.87s, avg_batch_time=0.0022s
Epoch 3: train=0.0425, val=0.0451, epoch_time=1.88s, avg_batch_time=0.0023s
Epoch 4: train=0.0450, val=0.3014, epoch_time=1.88s, avg_batch_time=0.0024s
Epoch 5: train=0.1442, val=0.0277, epoch_time=1.89s, avg_batch_time=0.0024s
Epoch 6: train=0.0252, val=0.0285, epoch_time=1.87s, avg_batch_time=0.0023s
Epoch 7: train=0.0273, val=0.0303, epoch_time=1.88s, avg_batch_time=0.0023s
Epoch 8: train=0.0299, val=0.0308, epoch_time=1.87s, avg_batch_time=0.0022s
Epoch 9: train=0.0332, val=0.0389, epoch_time=1.87s, avg_batch_time=0.0021s
Epoch 10: train=0.0355, val=0.0414, epoch_time=1.88s, avg_batch_time=0.0024s
Epoch 11: train=0.0345, val=0.0462, epoch_time=1.88s, avg_batch_time=0.0024s
Epoch 12: train=0.2241, val=0.1932, epoch_time=1.88s, avg_batch_time=0.0024s
Epoch 13:

Allow the file to be run as a standalone script (important for Windows).

In [263]:
import matplotlib.pyplot as plt

@torch.no_grad()
def save_visual_results(
    model,
    device,
    output_path="model_outputs.png",
    n_images=5
):
    model.eval()
    model.to(device)

    dataset = CustomDataset(train=False)

    fig, axs = plt.subplots(n_images, 3, figsize=(9, 3 * n_images))

    for i in range(n_images):
        x, y = dataset[i]

        pred = model(x.unsqueeze(0).to(device)).cpu().squeeze(0)

        # Input (RGB)
        axs[i, 0].imshow(x.permute(1, 2, 0))
        axs[i, 0].set_title("Input")
        axs[i, 0].axis("off")

        # Ground truth
        axs[i, 1].imshow(y.squeeze(), cmap="gray")
        axs[i, 1].set_title("Ground Truth")
        axs[i, 1].axis("off")

        # Model output
        axs[i, 2].imshow(pred.squeeze(), cmap="gray")
        axs[i, 2].set_title("Model Output")
        axs[i, 2].axis("off")

    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

    print(f"Saved visual comparison to: {output_path}")


In [264]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = TinyTransformNet()
    model.load_state_dict(
        torch.load("tiny_transform_net.pt", map_location=device)
    )

    save_visual_results(
        model,
        device,
        output_path="model_visual_results.png",
        n_images=5
    )

    test_inference_time(model, device)

freeze_support()
main()


Saved visual comparison to: model_visual_results.png
Sequential transforming each image took: 1.3164067s on CPU.
Using a model with batch_size=100 took 0.3160997s on cuda.
Using a model with batch_size=100 took 0.3160997s on cuda and NO DATA LOADER.

